In [ ]:
# The rest of the code needs this package update

!pip install datasets==4.0.0

import datasets
print(datasets.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

2.14.4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
import os

# Load fine-tuned BERT-tiny (distilled)
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Postdoc/results/student/tiny-v2/checkpoint-epoch6")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
# Display layers to prune

for layer_idx, layer in enumerate(model.bert.encoder.layer):
    total_heads = layer.attention.self.num_attention_heads
    print(f"Layer {layer_idx}: {total_heads} attention heads")

Layer 0: 2 attention heads
Layer 1: 2 attention heads


In [ ]:
# Choose heads to prune manually
heads_to_prune = {
    0: [1],  # remove head 1 from layer 0
    1: [0]   # remove head 0 from layer 1
}

# Apply head pruning
model.bert.prune_heads(heads_to_prune)

# Save the pruned model
save_path = "/content/drive/MyDrive/Colab Notebooks/Postdoc/results/student/tiny-v2/checkpoint-epoch6-pruned"
model.save_pretrained(save_path)

In [ ]:
# Displying pruned layers
for layer_idx, layer in enumerate(model.bert.encoder.layer):
    total_heads = layer.attention.self.num_attention_heads
    print(f"Layer {layer_idx}: {total_heads} attention heads")

Layer 0: 1 attention heads
Layer 1: 1 attention heads


In [ ]:
import pandas as pd
from datasets import Dataset

# Load and prepare dataset

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Postdoc/cleaned_stripped_mimic_notes.csv")
print(df.shape)

dataset = Dataset.from_pandas(df[["clean_relevant_note_truncate", "label"]])

# Split data into train and temp
temp_split = dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = temp_split["train"]
temp_dataset = temp_split["test"]

# Then split temp into val and test
val_test_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
val_dataset = val_test_split["train"]
test_dataset = val_test_split["test"]

(51695, 4)


In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

# Function to evaluate model on test set

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate_model_on_test_set_raw(model, test_dataset, tokenizer_name="gaunernst/bert-tiny-uncased", batch_size=8):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # Create DataLoader that tokenizes on-the-fly
    def collate_fn(batch):
        texts = [x["clean_relevant_note_truncate"] for x in batch]
        labels = torch.tensor([x["label"] for x in batch])
        encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
        return encodings, labels

    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # Move model to device
    model.eval()
    model.to(device)

    all_logits = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs)
            logits = outputs.logits

            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())

    # Concatenate results
    logits = torch.cat(all_logits, dim=0)
    labels = torch.cat(all_labels, dim=0)
    probs = torch.nn.functional.softmax(logits, dim=1)
    preds = torch.argmax(probs, dim=1).numpy()

    # Compute metrics
    labels_np = labels.numpy()
    acc = accuracy_score(labels_np, preds)
    prec = precision_score(labels_np, preds)
    rec = recall_score(labels_np, preds)
    f1 = f1_score(labels_np, preds)
    auc = roc_auc_score(labels_np, probs[:, 1].numpy())
    cm = confusion_matrix(labels_np, preds)

    # Display results
    print("🔍 Test Set Evaluation:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"AUROC:     {auc:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    print("\nDetailed Classification Report:")
    print(classification_report(labels_np, preds, digits=4))

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "auroc": auc,
        "confusion_matrix": cm
    }


In [ ]:
# Evaluate pruned model on test set before retraining

print("Pruned model:")
results = evaluate_model_on_test_set_raw(model, test_dataset, tokenizer_name="gaunernst/bert-tiny-uncased")

Pruned model:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Evaluating: 100%|██████████| 970/970 [00:19<00:00, 49.51it/s]

🔍 Test Set Evaluation:
Accuracy:  0.7474
Precision: 0.5872
Recall:    0.6071
F1 Score:  0.5970
AUROC:     0.7836

Confusion Matrix:
[[4345 1020]
 [ 939 1451]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.8223    0.8099    0.8160      5365
           1     0.5872    0.6071    0.5970      2390

    accuracy                         0.7474      7755
   macro avg     0.7048    0.7085    0.7065      7755
weighted avg     0.7498    0.7474    0.7485      7755



In [ ]:
from transformers import TrainingArguments

# Arguments for retraining

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/Postdoc/results/student/pruned-tiny/v1",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Colab Notebooks/Postdoc/logs/student/pruned-tiny/v1",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [ ]:
from transformers import Trainer

# Custom class to give label = 0 or 1 different weights in the cross entropy loss function, to account for class imbalance
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Setting the weights here
        weights = torch.tensor([0.72, 1.60]).to(logits.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

tokenizer = AutoTokenizer.from_pretrained("gaunernst/bert-tiny-uncased")

In [ ]:
import numpy as np
!pip install evaluate
import evaluate

# Metrics to compute

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"],
        "recall": recall.compute(predictions=preds, references=labels)["recall"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"]
    }

In [ ]:
# Tokenize all examples

def tokenize_batch(example):
    return tokenizer(
        example["clean_relevant_note_truncate"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize_batch, batched=True)

tokenized_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'label']
)

# Split data into train and temp
temp_split = tokenized_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = temp_split["train"]
temp_dataset = temp_split["test"]

# Then split temp into val and test
val_test_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
val_dataset = val_test_split["train"]
test_dataset = val_test_split["test"]

Map:   0%|          | 0/51695 [00:00<?, ? examples/s]

In [ ]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-13-1845219473.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedLossTrainer(


In [ ]:
# Train model on 5 epochs

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: lili-zeng (lili-zeng-mcgill-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,0.473300,0.557937,0.759479,0.626028,0.652047,0.602005
2,0.414200,0.574647,0.752515,0.624829,0.667502,0.587284
3,0.401600,0.573316,0.758834,0.621304,0.640769,0.602987
4,0.450700,0.600091,0.764638,0.625487,0.636591,0.614764
5,0.409200,0.616230,0.765025,0.622462,0.627402,0.617599


TrainOutput(global_step=22620, training_loss=0.46328381463345125, metrics={'train_runtime': 464.0467, 'train_samples_per_second': 389.896, 'train_steps_per_second': 48.745, 'total_flos': 193229766144000.0, 'train_loss': 0.46328381463345125, 'epoch': 5.0})

In [ ]:
# Save model and tokenizer to a folder
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-pruned/v1")           # saves model weights + config
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-pruned/v1")    # saves tokenizer files

('/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-pruned/v1/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-pruned/v1/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-pruned/v1/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-pruned/v1/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model-pruned/v1/tokenizer.json')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import torch

# Function to evaluate the model on the test set
def evaluate_model_on_test_set(trainer, test_dataset):
    # Ensure test dataset is in torch format
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # Run model predictions
    predictions = trainer.predict(test_dataset)
    logits = predictions.predictions
    labels = predictions.label_ids
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1)
    preds = torch.argmax(probs, axis=1).numpy()

    # Compute metrics
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    rec = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    auc = roc_auc_score(labels, probs[:, 1])
    cm = confusion_matrix(labels, preds)

    # Display results
    print("🔍 Test Set Evaluation:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"AUROC:     {auc:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    print("\nDetailed Classification Report:")
    print(classification_report(labels, preds, digits=4))

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "auroc": auc,
        "confusion_matrix": cm
    }

In [ ]:
# Evaluate model after retraining.

print("Pruned model retrained:")
results = evaluate_model_on_test_set(trainer, test_dataset)

Pruned model retrained:


🔍 Test Set Evaluation:
Accuracy:  0.7631
Precision: 0.6085
Recall:    0.6490
F1 Score:  0.6281
AUROC:     0.8106

Confusion Matrix:
[[4367  998]
 [ 839 1551]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.8388    0.8140    0.8262      5365
           1     0.6085    0.6490    0.6281      2390

    accuracy                         0.7631      7755
   macro avg     0.7237    0.7315    0.7271      7755
weighted avg     0.7678    0.7631    0.7652      7755

